# 타이타닉 데이터를 활용하여 생사여부 예측
- 단 train과 validation_set비율은 7:3으로 한다.


In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

### 데이터 탐색

In [31]:
titanic_train = pd.read_csv('../data/Titanic_train.csv', index_col='PassengerId')
titanic_train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [32]:
train_feature = titanic_train.iloc[:,1:]
train_target = titanic_train.iloc[:,0]

In [33]:
train_feature.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Name      891 non-null    object 
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Ticket    891 non-null    object 
 7   Fare      891 non-null    float64
 8   Cabin     204 non-null    object 
 9   Embarked  889 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 76.6+ KB


In [34]:
def cort(name : str):
    name_splist = name.split()
    
    for chunck in name_splist:
        if chunck.endswith('.'):
            return chunck
    return ''

In [35]:
train_feature['Embarked'] = train_feature['Embarked'].fillna("S")

In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler

### Name

In [37]:
def cort(name : str):
    name_splist = name.split()
    
    for chunck in name_splist:
        if chunck.endswith('.'):
            return chunck
    return ''

In [38]:
train_feature['FirstName'] =  train_feature['Name'].map(cort)

### SibSp쪽 

In [39]:
from sklearn.preprocessing import OneHotEncoder

encoderP = OneHotEncoder(sparse_output=False)
# encoderE = OneHotEncoder()
# encoderF = OneHotEncoder()
# encoderS = OneHotEncoder()

In [40]:
titanic_train = pd.read_csv('../data/Titanic_train.csv', index_col='PassengerId')

train_feature = titanic_train.iloc[:,1:]
train_feature['Embarked'] = train_feature['Embarked'].fillna("S")
train_feature['FirstName'] =  train_feature['Name'].map(cort)

train_feature['FamilySize']= train_feature['SibSp'] + train_feature['Parch'] +1
train_feature.drop(['SibSp', 'Parch'], axis=1, inplace=True)


train_feature['Age'] = train_feature['Age'].fillna(train_feature['Age'].median())
train_feature['Fare'] = train_feature['Fare'].fillna(train_feature['Fare'].median())


train_feature['CabinNA'] = train_feature['Cabin'].isna()

train_feature.drop(['Name', 'Cabin', 'Ticket'], axis=1, inplace=True)


temp = pd.DataFrame( encoderP.fit_transform(train_feature.loc[:, ['Pclass', 'Embarked', 'FirstName', 'Sex']].astype(str)), index=range(1, 892))



In [41]:
train_feature_svm = pd.concat([train_feature, temp], axis = 1)
train_feature_svm = train_feature_svm.drop(['Pclass', 'Embarked', 'FirstName', 'Sex'], axis=1)

In [42]:
train_feature_svm

,Age,Fare,FamilySize,CabinNA,0,1,2,3,4,5,...,15,16,17,18,19,20,21,22,23,24
1,22.0,7.2500,2,True,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,38.0,71.2833,2,False,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,26.0,7.9250,1,True,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,35.0,53.1000,2,False,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
5,35.0,8.0500,1,True,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,27.0,13.0000,1,True,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
888,19.0,30.0000,1,False,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
889,28.0,23.4500,4,True,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
890,26.0,30.0000,1,False,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [43]:
stds = StandardScaler()
train_feature_svm =  stds.fit_transform(train_feature_svm)
svm_train_featrue, svm_valid_featrue, train_target_train, train_target_valid = train_test_split( train_feature_svm,
                                                                                                train_target,
                                                                                                stratify=train_target,
                                                                                                test_size=0.2
)

TypeError: Feature names are only supported if all input features have string names, but your input has ['int', 'str'] as feature name / column name types. If you want feature names to be stored and validated, you must convert them all to strings, by using X.columns = X.columns.astype(str) for example. Otherwise you can remove feature / column names from your input data, or convert them all to a non-string data type.

In [44]:
svm_train_featrue.shape

(712, 29)

### 쓸모없는거 드랍후 확인

In [45]:
train_feature.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      891 non-null    int64  
 1   Sex         891 non-null    object 
 2   Age         891 non-null    float64
 3   Fare        891 non-null    float64
 4   Embarked    891 non-null    object 
 5   FirstName   891 non-null    object 
 6   FamilySize  891 non-null    int64  
 7   CabinNA     891 non-null    bool   
dtypes: bool(1), float64(2), int64(2), object(3)
memory usage: 56.6+ KB


### Tree계열 인코딩

In [46]:
train_feature_tree = train_feature.copy()

In [47]:
def embark_encoder(x):
    if x == 'S':
        return 1
    if x == 'C':
        return 2
    if x == 'Q':
        return 3


train_feature_tree['Embarked'] = train_feature_tree['Embarked'].map(embark_encoder)
train_feature_tree['Sex'] = train_feature_tree['Sex'].map(lambda x : x == 'male')
def age_encdoer(age):
    if age < 10:
        return 1
    if age < 20:
        return 2
    if age < 30:
        return 3
    if age < 40:
        return 4
    if age >= 40:
        return 5


train_feature_tree['Age'] =  train_feature_tree['Age'].map(age_encdoer)


In [48]:
def firstname_encoder(x):
    if x == 'Mr.':
        return 1
    if x == 'Miss.':
        return 2
    if x == 'Mrs.':
        return 3
    if x == 'Master':
        return 4
    else:
        return 5

train_feature_tree['FirstName'] = train_feature_tree['FirstName'].map(firstname_encoder)
# train_feature_tree.drop('FirstName', axis=1, inplace=True)

In [49]:
tree_train_featrue, tree_valid_featrue, tree_target_train, tree_target_valid = train_test_split( train_feature_tree,
                                                                                                train_target,
                                                                                                stratify=train_target,
                                                                                                test_size=0.2
)

### svm계열 인코딩

In [50]:
pd.cut(train_feature['Pclass'], 3)

PassengerId
1        (2.333, 3.0]
2      (0.998, 1.667]
3        (2.333, 3.0]
4      (0.998, 1.667]
5        (2.333, 3.0]
            ...      
887    (1.667, 2.333]
888    (0.998, 1.667]
889      (2.333, 3.0]
890    (0.998, 1.667]
891      (2.333, 3.0]
Name: Pclass, Length: 891, dtype: category
Categories (3, interval[float64, right]): [(0.998, 1.667] < (1.667, 2.333] < (2.333, 3.0]]

### 그리드서치


In [51]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC



### tree

In [52]:
model = RandomForestClassifier()

# 파라미터 그리드 정의


In [53]:
param_grid = {
    'n_estimators': [100, 200, 300],  # 트리의 개수
    'max_depth': [None, 10, 20, 30],  # 트리의 최대 깊이
    'min_samples_split': [2, 5, 10],  # 내부 노드를 분할하는데 필요한 최소 샘플 수
    'min_samples_leaf': [1, 2, 4],    # 리프 노드에 있어야 할 최소 샘플 수
    'max_features': ['auto', 'sqrt'],  # 최적의 분할을 위해 고려할 특성의 수
    'bootstrap': [True, False]        # 부트스트랩 샘플 사용 여부
}

In [54]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=2)
grid_search.fit(tree_train_featrue, tree_target_train)
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)

KeyboardInterrupt: 

In [55]:
from sklearn.metrics import classification_report

y_pred =  grid_search.best_estimator_.predict(tree_valid_featrue)




print(classification_report(y_pred, tree_target_valid))

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [ ]:
svm_train_featrue.shape

(712, 30)

### svc


In [56]:
model = SVC()
param_grid = {
    'C': [0.1, 1, 10, 100], 
    'kernel': ['rbf', 'linear'],
    'gamma': ['scale', 'auto', 0.1, 1]
}

In [57]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(svm_train_featrue, train_target_train)
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)# 

Best parameters: {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}
Best cross-validation score: 0.8371023342854329


In [58]:
grid_search.best_estimator_.fit(svm_train_featrue, train_target_train)

SVC(C=1, kernel='linear')

In [60]:
y_pred = grid_search.best_estimator_.predict(svm_valid_featrue)
# accuracy = accuracy_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)
# print(f"Test set accuracy: {accuracy * 100:.2f}%")
# print(f"Test set f1: {f1 * 100:.2f}%")

from sklearn import metrics
print(metrics.classification_report(train_target_valid,y_pred))

              precision    recall  f1-score   support

           0       0.83      0.89      0.86       110
           1       0.80      0.71      0.75        69

    accuracy                           0.82       179
   macro avg       0.82      0.80      0.81       179
weighted avg       0.82      0.82      0.82       179



In [61]:
titanic_train = pd.read_csv('../data/competition_test.csv', index_col='PassengerId')

train_feature = titanic_train.iloc[:,:]
train_feature['Embarked'] = train_feature['Embarked'].fillna("S")
train_feature['FirstName'] =  train_feature['Name'].map(cort)

train_feature['FamilySize']= train_feature['SibSp'] + train_feature['Parch'] +1
train_feature.drop(['SibSp', 'Parch'], axis=1, inplace=True)


train_feature['Age'] = train_feature['Age'].fillna(train_feature['Age'].median())
train_feature['Fare'] = train_feature['Fare'].fillna(train_feature['Fare'].median())


train_feature['CabinNA'] = train_feature['Cabin'].isna()

train_feature.drop(['Name', 'Cabin', 'Ticket'], axis=1, inplace=True)

temp = pd.DataFrame( encoderP.transform(train_feature.loc[:, ['Pclass', 'Embarked', 'FirstName', 'Sex']].astype(str)))
train_feature_svm = pd.concat([train_feature, temp], axis = 1)
train_feature_svm = train_feature_svm.drop(['Pclass', 'Embarked', 'FirstName', 'Sex'], axis=1)

# stds = StandardScaler()
train_feature_svm =  stds.transform(train_feature_svm)

# svm_train_featrue, svm_valid_featrue, train_target_train, train_target_valid = train_test_split( train_feature_svm,
#                                                                                                 train_target,
#                                                                                                 stratify=train_target,
#                                                                                                 test_size=0.2
# )


ValueError: Found unknown categories ['Dona.'] in column 2 during transform

In [ ]:
train_feature_svm.shape

(407, 21)

In [ ]:
grid_search.best_estimator_.predict(train_feature_svm)

ValueError: X has 21 features, but SVC is expecting 30 features as input.

In [ ]:
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'metric': ['euclidean', 'manhattan']
}

# GridSearchCV 설정
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, scoring='accuracy')

# 모델 학습 (X_train과 y_train은 학습 데이터)
grid_search.fit(svm_train_featrue, train_target_train)

# 최적의 파라미터와 점수 출력
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)

Best parameters: {'algorithm': 'auto', 'metric': 'manhattan', 'n_neighbors': 11, 'weights': 'uniform'}
Best cross-validation score: 0.8201418300009848


In [ ]:
y_pred = grid_search.best_estimator_.predict(svm_valid_featrue)
# accuracy = accuracy_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)
# print(f"Test set accuracy: {accuracy * 100:.2f}%")
# print(f"Test set f1: {f1 * 100:.2f}%")

from sklearn import metrics
print(metrics.classification_report(train_target_valid,y_pred))

              precision    recall  f1-score   support

           0       0.80      0.89      0.84       110
           1       0.79      0.65      0.71        69

    accuracy                           0.80       179
   macro avg       0.80      0.77      0.78       179
weighted avg       0.80      0.80      0.79       179



### 민철님꺼

In [ ]:
train_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,AgeGroup,FamilySize,IsAlone,Title,Deck,FareBin,AgeBin
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,2,0,Mr,NaN,"(-0.001, 7.91]","(20.125, 28.0]"
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,3,2,0,Mrs,C,"(31.0, 512.329]","(28.0, 38.0]"
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,2,1,1,Miss,NaN,"(7.91, 14.454]","(20.125, 28.0]"
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,3,2,0,Mrs,C,"(31.0, 512.329]","(28.0, 38.0]"
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,3,1,1,Mr,NaN,"(7.91, 14.454]","(28.0, 38.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,2,1,1,Rare,NaN,"(7.91, 14.454]","(20.125, 28.0]"
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1,1,1,Miss,B,"(14.454, 31.0]","(0.419, 20.125]"
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,NaN,4,0,Miss,NaN,"(14.454, 31.0]",NaN
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,2,1,1,Mr,C,"(14.454, 31.0]","(20.125, 28.0]"


In [ ]:
trdata = train_data
trdata = pd.get_dummies(trdata, columns=['Sex'], drop_first=True)
trdata = trdata.drop(columns=['Ticket', 'Cabin', 'Name', 'Deck', 'PassengerId', 'Age', 'Embarked', 'FareBin', 'AgeBin'])
trdata['Title'] = trdata['Title'].map({'Mr': 1, 'Mrs': 2, 'Miss': 3, 'Master': 4, 'Rare': 5})
trdata.corr()

,Survived,Pclass,SibSp,Parch,Fare,AgeGroup,FamilySize,IsAlone,Title,Sex_male
Survived,1.000000,-0.338481,-0.035322,0.081629,0.257307,-0.066454,0.016639,-0.203367,0.403967,-0.543351
Pclass,-0.338481,1.000000,0.083081,0.018443,-0.549500,-0.374848,0.065997,0.135207,-0.116169,0.131900
SibSp,-0.035322,0.083081,1.000000,0.414838,0.159651,-0.285537,0.890712,-0.584471,0.269461,-0.114631
Parch,0.081629,0.018443,0.414838,1.000000,0.216225,-0.166530,0.783111,-0.583398,0.270647,-0.245489
Fare,0.257307,-0.549500,0.159651,0.216225,1.000000,0.105290,0.217138,-0.271832,0.140981,-0.182333
AgeGroup,-0.066454,-0.374848,-0.285537,-0.166530,0.105290,1.000000,-0.274693,0.170605,-0.238160,0.097072
FamilySize,0.016639,0.065997,0.890712,0.783111,0.217138,-0.274693,1.000000,-0.690922,0.319378,-0.200988
IsAlone,-0.203367,0.135207,-0.584471,-0.583398,-0.271832,0.170605,-0.690922,1.000000,-0.286865,0.303646
Title,0.403967,-0.116169,0.269461,0.270647,0.140981,-0.238160,0.319378,-0.286865,1.000000,-0.562998
Sex_male,-0.543351,0.131900,-0.114631,-0.245489,-0.182333,0.097072,-0.200988,0.303646,-0.562998,1.000000


In [ ]:
trdata = trdata[['Survived','Pclass', 'Fare', 'IsAlone', 'Title', 'Sex_male']]

In [ ]:
trdata

,Survived,Pclass,Fare,IsAlone,Title,Sex_male
0,0,3,7.2500,0,1,True
1,1,1,71.2833,0,2,False
2,1,3,7.9250,1,3,False
3,1,1,53.1000,0,2,False
4,0,3,8.0500,1,1,True
...,...,...,...,...,...,...
886,0,2,13.0000,1,5,True
887,1,1,30.0000,1,3,False
888,0,3,23.4500,0,3,False
889,1,1,30.0000,1,1,True


In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],  # 트리의 개수
    'max_depth': [None, 10, 20, 30],  # 트리의 최대 깊이
    'min_samples_split': [2, 5, 10],  # 내부 노드를 분할하는데 필요한 최소 샘플 수
    'min_samples_leaf': [1, 2, 4],    # 리프 노드에 있어야 할 최소 샘플 수
    'max_features': ['auto', 'sqrt'],  # 최적의 분할을 위해 고려할 특성의 수
    'bootstrap': [True, False]        # 부트스트랩 샘플 사용 여부
}

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# 3. 특성과 레이블 분리
X = trdata.drop('Survived', axis=1)
y = trdata['Survived']

# 4. 훈련/검증 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 5. 모델 훈련 (교차검증 포함)
model = RandomForestClassifier(n_estimators=200, random_state=42)

# 교차검증 수행 (cv=5는 5개의 폴드로 교차검증을 수행)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)


# 6. 모델 학습 후 예측 및 평가
# model.fit(X_train, y_train)
y_pred = grid_search.best_estimator_.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"Test set accuracy: {accuracy * 100:.2f}%")
print(f"Test set f1: {f1 * 100:.2f}%")

from sklearn import metrics
print(metrics.classification_report(y_test,y_pred))

Best parameters: {'bootstrap': False, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}
Best cross-validation score: 0.8187096774193549
Test set accuracy: 82.46%
Test set f1: 77.29%
              precision    recall  f1-score   support

           0       0.82      0.90      0.86       157
           1       0.83      0.72      0.77       111

    accuracy                           0.82       268
   macro avg       0.83      0.81      0.82       268
weighted avg       0.83      0.82      0.82       268



In [ ]:
print(f"Test set accuracy: {accuracy * 100:.2f}%")
print(f"Test set f1: {f1 * 100:.2f}%")

Test set accuracy: 82.46%
Test set f1: 77.07%


Best parameters: {'bootstrap': True, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
Best cross-validation score: 0.7407917428795401
Test set accuracy: 82.46%
              precision    recall  f1-score   support

           0       0.82      0.90      0.86       157
           1       0.84      0.71      0.77       111

    accuracy                           0.82       268
   macro avg       0.83      0.81      0.81       268
weighted avg       0.83      0.82      0.82       268

Best parameters: {'bootstrap': True, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}
Best cross-validation score: 0.7586340397599957
Test set accuracy: 82.09%
              precision    recall  f1-score   support

           0       0.80      0.92      0.86       157
           1       0.86      0.68      0.76       111

    accuracy                           0.82       268
   macro avg       0.83      0.80      0.81       268
weighted avg       0.83      0.82      0.82       268





In [ ]:
%pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectFromModel
from imblearn.over_sampling import SMOTE

# 1단계: 데이터 로드 및 특성 엔지니어링  1
def feature_engineering(df):
    df['AgeGroup'] = pd.cut(df['Age'], bins=[0, 14, 24, 34, 44, 54, 64, np.inf],
                            labels=['0-14', '15-24', '25-34', '35-44', '45-54', '55-64', '65+'])
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
    df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    df['Title'] = df['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    df['Title'] = df['Title'].replace(['Mlle', 'Ms', 'Mme'], ['Miss', 'Miss', 'Mrs'])
    df['Deck'] = df['Cabin'].str[0].fillna('Unknown')
    
    # FareBin과 AgeBin을 숫자로 변환
    df['FareBin'] = pd.qcut(df['Fare'], 4, labels=False)
    # df['AgeBin'] = pd.qcut(df['Age'], 4, labels=False)
    
    return df


train_data = feature_engineering(pd.read_csv("../Data/Titanic_train.csv"))
test_data = feature_engineering(pd.read_csv("../Data/competition_test.csv"))


# 2단계: 레이블 인코딩
le = LabelEncoder()
categorical_features = ['Pclass', 'Sex', 'Embarked', 'AgeGroup', 'Title', 'Deck']
for feature in categorical_features:
    train_data[feature] = le.fit_transform(train_data[feature].astype(str))
    test_data[feature] = le.transform(test_data[feature].astype(str))


# 3단계: 결측값 처리
numeric_features = ['Age', 'Fare']
imputer = SimpleImputer(strategy='median')
train_data[numeric_features] = imputer.fit_transform(train_data[numeric_features])
test_data[numeric_features] = imputer.transform(test_data[numeric_features])


# 4단계: 특성과 목표 변수 설정
X = train_data.drop(columns=['Survived', 'Name', 'Ticket', 'Cabin'])
y = train_data['Survived']


# 특성 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


# 6단계: 불균형 데이터 처리
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# 7단계: Train/Test Split
X_train, X_val, y_train, y_val = train_test_split(X_resampled, y_resampled, test_size=0.3, stratify=y_resampled)

# 8단계: 모델 생성 및 하이퍼파라미터 튜닝
rf_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


rf_grid = GridSearchCV(RandomForestClassifier(random_state=42), rf_params, cv=10, n_jobs=-1, scoring='f1')
rf_grid.fit(X_train, y_train)


gb_params = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

gb_grid = GridSearchCV(GradientBoostingClassifier(random_state=42), gb_params, cv=10, n_jobs=-1, scoring='f1')
gb_grid.fit(X_train, y_train)

# 9단계: 앙상블 모델 생성
ensemble = VotingClassifier(
    estimators=[('rf', rf_grid.best_estimator_), ('gb', gb_grid.best_estimator_)],
    voting='soft'
)
ensemble.fit(X_train, y_train)

# 10단계: 모델 평가
y_val_pred = ensemble.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
f1_accuracy = f1_score(y_val, y_val_pred)
print("Ensemble Validation Accuracy:", val_accuracy)
print("Ensemble Validation F1:", f1_accuracy)
print("\nEnsemble Classification Report:")
print(classification_report(y_val, y_val_pred))

# 11단계: 특성 중요도 출력 (Random Forest 기준)
# feature_importance = pd.DataFrame({
#     'feature': X.columns[selector.get_support()],
#     'importance': rf_grid.best_estimator_.feature_importances_
# })
# feature_importance = feature_importance.sort_values('importance', ascending=False)
# print("\nFeature Importance:")
# print(feature_importance.head(10)) 

Ensemble Validation Accuracy: 0.8606060606060606
Ensemble Validation F1: 0.8647058823529412

Ensemble Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.83      0.86       165
           1       0.84      0.89      0.86       165

    accuracy                           0.86       330
   macro avg       0.86      0.86      0.86       330
weighted avg       0.86      0.86      0.86       330



IndexError: boolean index did not match indexed array along dimension 0; dimension is 14 but corresponding boolean dimension is 15

In [ ]:
X.isna().sum()

PassengerId      0
Pclass           0
Sex              0
Age              0
SibSp            0
Parch            0
Fare             0
Embarked         0
AgeGroup         0
FamilySize       0
IsAlone          0
Title            0
Deck             0
FareBin          0
AgeBin         177
dtype: int64